In [72]:
#import libraries

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

from sklearn.impute import KNNImputer


from string import ascii_lowercase
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier
import lightgbm as lgb

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [73]:
#Data Load
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Q

In [74]:
Answers = []
for i in range(1, 27):
    Answers.append('Q' + str(i))
    
Q_Answers = []
for i in range(1, 21):
    Q_Answers.append('Q' + str(i))

In [75]:
def knull(col):
    imputer = KNNImputer(n_neighbors=3)
    a = imputer.fit_transform(train[col])
    train[col] = a

In [76]:
knull(Answers)

In [77]:
# Test data의 결측치 train의 최빈값으로 채우기
for i in range(len(Answers)):
    test[Answers[i]]=test[Answers[i]].fillna(train[Answers[i]].mode()[0])

In [78]:
# 1~20번 항목 더해서 Q_score, 21~26번 항목 더해서 Qs_score 생성
Q_secret= ['Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']

dataset = [train, test]

for data in dataset:
    data['Q_score'] = data[Q_Answers].sum(axis=1)
    data['Qs_score'] = data[Q_secret].sum(axis=1)

In [79]:
# 마키아벨리즘 테스트(1~20번 항목)에서 T, V, M 컬럼 생성
for data in dataset:
    data['T'] = data['Q1']+data['Q2']+data['Q3']+data['Q7']+data['Q10']+data['Q12']+data['Q15']+data['Q16'] 
    data['V'] = data['Q4']+data['Q5']+data['Q8']+data['Q11']+data['Q13']+data['Q20']
    data['M'] = data['Q9']+data['Q19']

In [80]:
# 마키아벨리즘 스코어, 분산 피처 생성
for data in dataset:
    data['Mach_score'] = data[Answers].mean(axis = 1)
    data['Mach_var'] = data[Answers].var(axis = 1)

In [81]:
# 마키아벨리즘 비율 피처 생성
Ancoms = list(combinations(Answers, 2))
for data in dataset:
    for a,b in Ancoms:
        data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]

C:\Users\thdal\AppData\Local\Temp/ipykernel_25216/4282676114.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]


In [82]:
# Q drop
for data in dataset:
    data.drop([('Q'+str(i) )for i in range(1,27)], axis = 1, inplace = True)

# TIPI

In [83]:
TIPI = []
for i in range(1,11):
    TIPI.append('TIPI'+str(i))

In [84]:
knull(TIPI)

In [85]:
for i in range(len(TIPI)):
    test[TIPI[i]] = test[TIPI[i]].fillna(train[TIPI[i]].mode()[0])

In [86]:
# tp1~10 항목 사용해 Ex, Ag, Con, Es, Op 컬럼 만들기

train['Extraverted'] = train['TIPI1'] - train['TIPI6']
train['Warm'] = train['TIPI7']-train['TIPI2']
train['Dependable'] = train['TIPI3']-train['TIPI8']
train['Calm'] = train['TIPI9']-train['TIPI4']
train['OpenMind'] = train['TIPI5']-train['TIPI10']

test['Extraverted'] = test['TIPI1'] - test['TIPI6']
test['Warm'] = test['TIPI7']-test['TIPI2']
test['Dependable'] = test['TIPI3']-test['TIPI8']
test['Calm'] = test['TIPI9']-test['TIPI4']
test['OpenMind'] = test['TIPI5']-test['TIPI10']

In [87]:
# tp 비율 피처 생성
tpcoms = list(combinations(TIPI, 2))

for data in dataset:
    for a,b in tpcoms:
        data['%s_dv_%s'%(a,b)] = data[a]/data[b]

In [88]:
train.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
train.drop('TIPI10', axis = 1, inplace = True)

test.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
test.drop('TIPI10', axis = 1, inplace = True)

In [89]:
#index drop
for data in dataset:
    data.drop('index', axis=1, inplace=True)

# Country

In [90]:
def country_label(data):
    nara = data['country'].copy()
    nara_val = nara.value_counts()
    
    a = [] 
    b = []
    c = [] 
    d = []
    e = []

    
    for i in range(len(nara_val)):
        if nara_val.values[i] <= 10 : a.append(nara_val.index[i])
        elif 100>nara_val.values[i] >10 : b.append(nara_val.index[i])
        elif 500>nara_val.values[i] >= 100 :c.append(nara_val.index[i])
        elif 5000>nara_val.values[i] >= 500 :  d.append(nara_val.index[i])
        elif nara_val.values[i] >= 5000 : e.append(nara_val.index[i])

    for i in range(len(nara)):
        if nara[i] in a: nara[i]=0
        if nara[i] in b: nara[i]=1
        if nara[i] in c: nara[i]=2
        if nara[i] in d: nara[i]=3
        if nara[i] in e: nara[i]=4

        
    nara.fillna(0, inplace = True)
    return nara

In [91]:
train['nara'] = country_label(train)
test['nara'] = country_label(test)

In [92]:
train = train.drop(columns=['country'], axis=1)
test = test.drop(columns=['country'], axis=1)

# VCL

In [93]:
VCL = []
for i in range(1,17):
    VCL.append('VCL'+str(i))

In [94]:
#작성자의 영어 실력

# 실제로 있고 진짜 쉬운단어
VCL_1 = ['VCL1','VCL4','VCL5','VCL10','VCL14','VCL16']
# 실제로 있는데 어려운 단어
VCL_2 = ['VCL2','VCL3','VCL7','VCL8','VCL11','VCL13','VCL14']
# 세상에 없는 단어
VCL_3 = ['VCL6','VCL9','VCL12']

In [95]:
dataset = [train,test]
for data in dataset:
    data['VCL_score'] = data[VCL].sum(axis=1)
    data['VCL_1_score'] = data[VCL_1].sum(axis=1)
    data['VCL_2_score'] = data[VCL_2].sum(axis=1)
    data['VCL_3_score'] = data[VCL_3].sum(axis=1)
    data.drop([('VCL'+str(i) )for i in range(1,17)], axis = 1, inplace = True)

# Hand

In [96]:
train.drop('hand', axis=1, inplace = True)
test.drop('hand', axis=1, inplace = True)

# Familysize

In [97]:
#이상치 제거
train = train.drop(train[train.familysize > 50].index)

# Age

In [98]:
#이상치 제거
train = train.drop(train[train.age > 100].index)

In [99]:
# age로 연령대 구간 나누기
def age_categorize(age):
    age = (age // 10)
    return age

train['age_cuts'] = age_categorize(train['age'])
test['age_cuts'] = age_categorize(test['age'])

# 기존의 age drop하기
train.drop('age', axis=1, inplace=True)
test.drop('age', axis=1, inplace = True)

In [101]:
train['age_cuts']

0        2
1        4
2        4
3        1
4        1
        ..
14995    1
14996    4
14997    2
14998    2
14999    2
Name: age_cuts, Length: 14993, dtype: int64

In [102]:
test['age_cuts']

0        1
1        3
2        1
3        2
4        1
        ..
35447    1
35448    1
35449    3
35450    1
35451    1
Name: age_cuts, Length: 35452, dtype: int64

# train 결측치 대체: KNN 

In [103]:
#train 데이터 결측치 대체: KNN
knull(['familysize','religion','orientation','married','voted','nerdiness'])

# elapse

In [104]:
elapse = ['introelapse', 'testelapse', 'surveyelapse']
train[elapse]=np.log1p(train[elapse])

#걸린시간 합
train['elapse_sum'] = train['testelapse']+train['surveyelapse']
test['elapse_sum'] = test['testelapse']+test['surveyelapse']

train.drop('introelapse', axis=1, inplace = True)
test.drop('introelapse', axis=1, inplace = True)

train.drop('testelapse', axis=1, inplace = True)
test.drop('testelapse', axis=1, inplace = True)

train.drop('surveyelapse', axis=1, inplace = True)
test.drop('surveyelapse', axis=1, inplace = True)

# train, test 결측치 대체: train의 최빈값

In [105]:
#ASD 결측치 최빈값 대체
train['ASD'] = train['ASD'].fillna(train['ASD'].mode()[0])
test['ASD'] = test['ASD'].fillna(train['ASD'].mode()[0])

#education 결측치 최빈값 대체
train['education'] = train['education'].fillna(train['education'].mode()[0])
test['education'] = test['education'].fillna(train['education'].mode()[0])

#engnat 결측치 최빈값 대체
train['engnat'] = train['engnat'].fillna(train['engnat'].mode()[0])
test['engnat'] = test['engnat'].fillna(train['engnat'].mode()[0])

#gender 결측치 최빈값 대체
train['gender'] = train['gender'].fillna(train['gender'].mode()[0])
test['gender'] = test['gender'].fillna(train['gender'].mode()[0])

#religion 결측치 최빈값 대체
test['religion'] = test['religion'].fillna(train['religion'].mode()[0])

#orientation 결측치 최빈값 대체
test['orientation'] = test['orientation'].fillna(train['orientation'].mode()[0])

#voted 결측치 최빈값 대체
test['voted'] = test['voted'].fillna(train['voted'].mode()[0])

#married 결측치 최빈값 대체
test['married'] = test['married'].fillna(train['married'].mode()[0])

#familysize 결측치 최빈값 대체
test['familysize'] = test['familysize'].fillna(train['familysize'].mode()[0])

In [106]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
needenco = ['gender', 'religion', 'orientation']
for i in needenco:
    train[i] = encoder.fit_transform(train[i])
    test[i] = encoder.transform(test[i])

In [107]:
train.isnull().sum()

education      0
urban          0
gender         0
engnat         0
religion       0
              ..
VCL_1_score    0
VCL_2_score    0
VCL_3_score    0
age_cuts       0
elapse_sum     0
Length: 400, dtype: int64

In [108]:
test.isnull().sum()

education      0
urban          0
gender         0
engnat         0
religion       0
              ..
VCL_1_score    0
VCL_2_score    0
VCL_3_score    0
age_cuts       0
elapse_sum     0
Length: 399, dtype: int64

# Model1: ExtraTreesRegressor + Tune

In [109]:
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [110]:
n_estimators = int(391.707369561944)
max_depth = int(65.33353663762796)
min_samples_split = int(5.797880430957836)
min_samples_leaf = int(3.813420188476544)
bootstrap = bootstrap = 0
    
assert type(n_estimators) == int
assert type(max_depth) == int
assert type(min_samples_split) == int
assert type(min_samples_leaf) == int

In [111]:
#Extra Trees Regressor
from sklearn.ensemble import ExtraTreesRegressor

xtree_tune = ExtraTreesRegressor(n_estimators=n_estimators,
                               max_depth=max_depth,
                               min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               #max_features=max_features,
                               bootstrap=bootstrap,
                               oob_score=bootstrap,
                               n_jobs=6,
                               random_state=42,
                               verbose=0)

xtree_tune.fit(train_x, train_y)
preds_tune = xtree_tune.predict(test)

In [112]:
preds_tune

array([0.16504689, 0.82630009, 0.88022165, ..., 0.96351236, 0.10741688,
       0.66389599])

# Model2: LGBM 4개

In [113]:
# features, target split
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split

In [114]:
def lgbm_rfe_30(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=30)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [115]:
import warnings

warnings.filterwarnings(action='ignore')
#warnings.filterwarnings(action='default')

In [116]:
lgbm_30 = lgbm_rfe_30(train_x, train_y)

399 0.8511013639499383
359 0.8515813319789425
323 0.8467658159110986
290 0.845912123525977
261 0.8540828090251265
234 0.847330433370533
210 0.8474530886676871
189 0.848031526442294
170 0.8491241951106029
153 0.8488305134131917
137 0.843926316989659
123 0.8503968157993843
110 0.8410067754095132
99 0.8449481565427099
89 0.8492497296400844
80 0.8438978125896162
72 0.8449760850962873
64 0.849699753652882
57 0.8428854745032461
51 0.8451554612702943
45 0.8487435605968995
40 0.8428425739415654


In [117]:
model1 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_1 = train_x[lgbm_30.iloc[7,2]]
model1.fit(x_train_1, train_y)

pred_y1 = model1.predict_proba(test[lgbm_30.iloc[7,2]])
pred_y1 = pred_y1[:,1]

In [118]:
def lgbm_rfe_500(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=500)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [119]:
lgbm_500 = lgbm_rfe_500(train_x, train_y)

399 0.8439497910655255
359 0.8463794698063993
323 0.8447571822308129
290 0.8405022856357692
261 0.8419614784639882
234 0.8484191644699391
210 0.8409615488978258
189 0.8434097020879591
170 0.845312570361672
153 0.8435765494578328
137 0.8413388317912941
123 0.8475341228851085
110 0.8455103047014009
99 0.8401610856510108
89 0.8405268220136919
80 0.8417394964096063
72 0.84127994448428
64 0.8451099287463585
57 0.8370002228480443
51 0.8427350960267234
45 0.844510952461778
40 0.8405242240442647


In [120]:
model2 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_2 = train_x[lgbm_500.iloc[14,2]]
model2.fit(x_train_2, train_y)

pred_y2 = model2.predict_proba(test[lgbm_500.iloc[14,2]])
pred_y2 = pred_y2[:,1]

In [121]:
def lgbm_rfe_913(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=913)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [122]:
lgbm_913 = lgbm_rfe_913(train_x, train_y)

399 0.8539730457785306
359 0.8588248980152668
323 0.8568700703530121
290 0.8596940631202652
261 0.851089588377724
234 0.8555589617821379
210 0.8498647901244831
189 0.84890152079357
170 0.8538901994201332
153 0.8570626087538868
137 0.8533772447899167
123 0.8479985820860193
110 0.8504779109092777
99 0.8482026670176812
89 0.8444494671853369
80 0.8490406564895545
72 0.8465933692892302
64 0.8414898026813354
57 0.8484962375629431
51 0.8479526846261407
45 0.842712002965149
40 0.8519567328398345


In [123]:
model3 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_3 = train_x[lgbm_913.iloc[7,2]]
model3.fit(x_train_3, train_y)

pred_y3 = model3.predict_proba(test[lgbm_913.iloc[7,2]])
pred_y3 = pred_y3[:,1]

In [124]:
def lgbm_rfe_8(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=8)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [125]:
lgbm_8 = lgbm_rfe_8(train_x, train_y)

399 0.8510170453305398
359 0.8453643684934198
323 0.8502385243538362
290 0.8490490614672608
261 0.8500854621330383
234 0.8499102877340015
210 0.8457965610680928
189 0.8484138388923924
170 0.8534778148802897
153 0.8480471213389987
137 0.8441390092135992
123 0.842383819169478
110 0.8490137394163074
99 0.846091485834996
89 0.8474386056807047
80 0.8424834675571269
72 0.8428131400326917
64 0.8410220535953253
57 0.8469696551993485
51 0.8450065532455509
45 0.8474742149028041
40 0.8383338444979731


In [126]:
model4 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_4 = train_x[lgbm_8.iloc[8,2]]
model4.fit(x_train_4, train_y)

pred_y4 = model4.predict_proba(test[lgbm_8.iloc[8,2]])
pred_y4 = pred_y4[:,1]

In [127]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)

In [ ]:
# LGBM + Extrees Regressor

pred_y_ensemble = (preds_tune)*(0.75) + (pred_all)*(0.25)

In [ ]:
submission = pd.DataFrame({
    "nerdiness" : pred_y_ensemble
})
submission.to_csv('sub_0826.csv', index=True)

In [ ]:
subb = pd.read_csv('sub_0826_.csv')

In [ ]:
subb.rename(columns = {'Unnamed: 0' : 'index'}, inplace = True)

In [ ]:
subb.set_index('index', inplace = True)
subb